In [1]:
import sys
import os
from datetime import datetime, timedelta

project_root = os.path.abspath("../..")

if project_root not in sys.path:
    sys.path.append(project_root)

os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

In [2]:
from datapipeline.utils.spark_session import get_spark_session
spark = get_spark_session("Analytics_Build_Layer")

In [3]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from delta.tables import DeltaTable

In [4]:
gold_path = os.path.join(project_root, "sanewsstorage/gold/articles_enriched")

clusters_path = os.path.join(project_root, "sanewsstorage/ml/clusters")
clusters_labeled_path = os.path.join(project_root, "sanewsstorage/ml/clusters_labeled")
cluster_ts_path = os.path.join(project_root, "sanewsstorage/ml/trend_time_series")
trending_path = os.path.join(project_root, "sanewsstorage/ml/trending_clusters")
events_path = os.path.join(project_root, "sanewsstorage/ml/events")

analytics_base = os.path.join(project_root, "sanewsstorage/analytics")

topic_daily_path = os.path.join(analytics_base, "topic_daily")
topic_realtime_path = os.path.join(analytics_base, "topic_realtime")
event_snapshot_path = os.path.join(analytics_base, "event_snapshot")
entity_daily_path = os.path.join(analytics_base, "entity_daily")
search_index_path = os.path.join(analytics_base, "search_index")

In [5]:
gold_df = spark.read.format("delta").load(gold_path)
clusters_df = spark.read.format("delta").load(clusters_path)
clusters_labeled_df = spark.read.format("delta").load(clusters_labeled_path)
cluster_ts = spark.read.format("delta").load(cluster_ts_path)
trending = spark.read.format("delta").load(trending_path)
events = spark.read.format("delta").load(events_path)

In [6]:
topic_daily = cluster_ts.select(
    "cluster_id",
    "cluster_label",
    "date",
    "article_count"
)

(
    topic_daily
    .write
    .format("delta")
    .mode("overwrite")
    .partitionBy("date")
    .save(topic_daily_path)
)

In [16]:
topic_daily.show(5)

+----------+--------------------+----------+-------------+
|cluster_id|       cluster_label|      date|article_count|
+----------+--------------------+----------+-------------+
|        13|paid, plans, avai...|2026-02-04|          128|
|         3|news, earnings, f...|2026-02-05|           81|
|        19|available, plans,...|2026-02-05|          259|
|        16|   chars, bowl, news|2026-02-03|          283|
|        18|chars, available,...|2026-02-08|          136|
+----------+--------------------+----------+-------------+
only showing top 5 rows



In [17]:
topic_daily.count()

156

In [7]:
topic_realtime = trending.select(
    "cluster_id",
    "cluster_label",
    F.col("last_24h").alias("article_count_24h"),
    "prev_24h",
    "growth_rate",
    "trend_score"
)

In [8]:
last_24h_articles = (
    gold_df
    .filter(F.col("published_at") >= F.current_timestamp() - F.expr("INTERVAL 24 HOURS"))
    .join(clusters_df, "bronze_hash")
)

window_spec = Window.partitionBy("cluster_id").orderBy(F.desc("published_at"))

In [9]:
top_headlines = (
    last_24h_articles
    .withColumn("rank", F.row_number().over(window_spec))
    .filter("rank <= 5")
    .groupBy("cluster_id")
    .agg(
        F.collect_list("title").alias("top_headlines"),
        F.collect_list("url").alias("top_urls")
    )
)

topic_realtime = topic_realtime.join(top_headlines, "cluster_id", "left")

In [10]:
topic_realtime.write.format("delta").mode("overwrite").save(topic_realtime_path)

In [18]:
topic_realtime.show(5)

+----------+-------------+-----------------+--------+-----------+-----------+-------------+--------+
|cluster_id|cluster_label|article_count_24h|prev_24h|growth_rate|trend_score|top_headlines|top_urls|
+----------+-------------+-----------------+--------+-----------+-----------+-------------+--------+
+----------+-------------+-----------------+--------+-----------+-----------+-------------+--------+



In [11]:
event_snapshot = (
    events
    .filter(
        (F.col("is_event") == 1) &
        (F.col("date") >= F.current_date() - F.expr("INTERVAL 7 DAYS"))
    )
    .withColumn(
        "severity_score",
        F.when(F.col("event_intensity") == "Viral", 2)
         .when(F.col("event_intensity") == "Trending", 1)
         .otherwise(0)
    )
    .orderBy(F.desc("severity_score"), F.desc("z_score"))
    .drop("severity_score", "is_event")
)

event_snapshot.write.format("delta").mode("overwrite").save(event_snapshot_path)

In [19]:
event_snapshot.show(5)

+----------+-------------+----+-------------+------------+------------+-------+---------------+
|cluster_id|cluster_label|date|article_count|baseline_avg|baseline_std|z_score|event_intensity|
+----------+-------------+----+-------------+------------+------------+-------+---------------+
+----------+-------------+----+-------------+------------+------------+-------+---------------+



In [12]:
entity_daily = (
    gold_df
    .filter(F.size("entities") > 0)
    .withColumn("date", F.to_date("published_at"))
    .withColumn("entity", F.explode("entities"))
    .select(
        "date",
        F.col("entity.entity").alias("entity_name"),
        F.col("entity.label").alias("entity_label")
    )
    .filter(F.col("entity_name").isNotNull())
    .groupBy("entity_name", "entity_label", "date")
    .count()
    .withColumnRenamed("count", "mention_count")
)

In [13]:
(
    entity_daily
    .write
    .format("delta")
    .mode("overwrite")
    .partitionBy("date")
    .save(entity_daily_path)
)

In [20]:
entity_daily.show()

+--------------------+------------+----------+-------------+
|         entity_name|entity_label|      date|mention_count|
+--------------------+------------+----------+-------------+
|Circling Levi's S...|         LOC|2026-02-07|            1|
|        U.S. Customs|         ORG|2026-02-07|            1|
|         Ben Affleck|         PER|2026-02-04|            6|
|            fit2work|        MISC|2026-02-04|            1|
|  BRUCK-MÜRZZUSCHLAG|        MISC|2026-02-04|            1|
|               Spurs|         ORG|2026-02-04|            5|
|Add todel.icio.us...|        MISC|2026-02-04|           11|
|               Anson|         PER|2026-02-04|            3|
|                Avro|         ORG|2026-02-04|            3|
|                 Two|        MISC|2026-02-04|           18|
|Was Truly Shocked...|        MISC|2026-02-06|            1|
|                ONLY|         ORG|2026-02-07|         1211|
|             Украины|         LOC|2026-02-04|            7|
|W izraelskich ata...|  

In [23]:
entity_daily.count()

119623

In [14]:
search_index = (
    gold_df
    .select("bronze_hash", "title", "published_at", "url")
    .join(clusters_df, on="bronze_hash", how="left")
    .join(clusters_labeled_df, on="cluster_id", how="left")
)

In [15]:
if DeltaTable.isDeltaTable(spark, search_index_path):

    delta_table = DeltaTable.forPath(spark, search_index_path)

    (
        delta_table.alias("t")
        .merge(
            search_index.alias("s"),
            "t.bronze_hash = s.bronze_hash"
        )
        .whenMatchedUpdateAll()
        .whenNotMatchedInsertAll()
        .execute()
    )

else:
    search_index.write.format("delta").mode("overwrite").save(search_index_path)

In [21]:
search_index.show(5)

+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+
|cluster_id|         bronze_hash|               title|       published_at|                 url|       cluster_label|        top_keywords|
+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+
|        14|-2133243635794753027|Officials praise ...|2026-02-03 22:43:58|https://whdh.com/...|available, plans,...|available, plans,...|
|        16| 4245083832015084286|Heightened securi...|2026-02-07 06:24:31|https://www.cbsne...|   chars, bowl, news|chars, bowl, news...|
|        11| 2109186896498655789|InventHelp Invent...|2026-02-03 18:45:00|https://www.prnew...|plans, paid, avai...|plans, paid, avai...|
|         4|-3987269010444925942|Jennifer Lopez je...|2026-02-04 18:50:34|https://economict...|    chars, news, new|chars, news, new,...|
|        11| 2990353637198350821|f

In [22]:
search_index.count()

27663

In [24]:
spark.stop()